## Script que faz clustering com HDBSCAN

O **HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise)** é um algoritmo de agrupamento não supervisionado baseado na densidade dos dados, que estende o DBSCAN ao introduzir uma abordagem hierárquica e dinâmica para identificar clusters. Ele ajusta automaticamente os parâmetros de densidade e pode lidar com clusters de diferentes tamanhos e densidades, além de identificar pontos como ruídos.

Ideal para lidar com formas de clusters arbitrárias e dados com ruídos, o HDBSCAN é amplamente utilizado em **análise geoespacial**, **segmentação de imagens**, **classificação de texto baseada em embeddings** e **detecção de anomalias**. No caso de **classificação de texto**, o HDBSCAN pode agrupar documentos semelhantes a partir de vetores gerados por modelos de embeddings como Word2Vec, GloVe ou transformers, permitindo a descoberta de tópicos ou agrupamentos de textos sem rótulos prévios.

Principais Vantagens do HDBSCAN:
- **Sem necessidade de ε fixo**: O HDBSCAN ajusta dinamicamente os clusters com base na densidade local.
- **Robustez a ruídos**: Identifica eficientemente pontos que não pertencem a clusters densos.
- **Suporte a clusters de diferentes densidades**: Ideal para dados complexos.

Embora o HDBSCAN elimine a necessidade de configurar parâmetros como o raio de vizinhança (**ε**), ele ainda exige a definição de valores como o tamanho mínimo do cluster (**min_cluster_size**) e o número mínimo de amostras (**min_samples**), que podem impactar os resultados.


Também será havaliado:
- **Proporção de ruído**: Portarias que não foram possíveis classificar
- **Total de clustar válidos gerados**

Adicionalmente, o teste levará em consideração:
- A análise com o texto original e o texto tratado.
- A aplicação de métodos de redução de dimensionalidade para otimizar os dados.

---

**1)** Execução Inicial:
O script inicia com a leitura do arquivo:  
**`f'./saida/06_2_DOU2_portarias_mgi_tratado_NER_lematizado_embeddingsUSE_SBERT_Reducao_Kmeans_DBSCAN.parquet'`**  
Caso este arquivo não exista, executar o notebook **`06_2_Clustering_DBSCAN.ipynb`** para gerar os dados necessários.



## Bibliotecas

In [1]:
# Bibliotecas para manipulação de dados e cálculo
import pandas as pd
import numpy as np
import time  # Para medir o tempo de execução

# Bibliotecas para visualização
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Bibliotecas para clusterização
from sklearn.cluster import DBSCAN

# Bibliotecas de aprendizado de máquina e redução de dimensionalidade
from sklearn.decomposition import PCA, KernelPCA
# from sklearn.manifold import TSNE, LocallyLinearEmbedding
# from sklearn.preprocessing import StandardScaler
# import umap

# Métricas para avaliação de clusters
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

# Hierarquia para dendrogramas
from scipy.cluster.hierarchy import dendrogram, linkage



In [2]:
douItem = 2
secao = f'Secao0{douItem}'

# Lê o arquivo Parquet
df_portarias_mgi = pd.read_parquet(f'./saida/06_2_DOU{secao}_portarias_mgi_tratado_NER_lematizado_embeddingsUSE_SBERT_Reducao_Kmeans_DBSCAN.parquet', engine='pyarrow')  # ou engine='fastparquet'

# Exibe as primeiras linhas do DataFrame
df_portarias_mgi.shape


(19258, 99)

In [3]:
df_portarias_mgi.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,USE_Embeddings4_PCA_TSNE_dbscan,USE_Embeddings4_UMAP_dbscan,SBERT_Embeddings1_dbscan,SBERT_Embeddings1_TSNE_dbscan,SBERT_Embeddings1_PCA_TSNE_dbscan,SBERT_Embeddings1_UMAP_dbscan,SBERT_Embeddings4_dbscan,SBERT_Embeddings4_TSNE_dbscan,SBERT_Embeddings4_PCA_TSNE_dbscan,SBERT_Embeddings4_UMAP_dbscan
0,30266027,PORTARIA DE PESSOAL 529,9368436,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,-1,0,0,-1,-1,0,-1,-1,0,0
1,30264410,ATO PORTARIA 655,9368530,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,-1,1,0,-1,-1,1,-1,0,-1,6
2,30265009,ATO PORTARIA DE PESSOAL 461,9368536,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,-1,1,0,-1,-1,1,-1,0,1,6
3,30265678,ATO PORTARIA DE PESSOAL SGP 606,9368541,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,110,2,0,0,0,2,-1,1,2,1
4,30266030,ATO PORTARIA DE PESSOAL SGP 517,9368573,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,0,-1,0,-1,968,2,-1,-1,3,1


In [4]:
df_portarias_mgi.columns

Index(['id', 'name', 'idOficio', 'pubName', 'artType', 'pubDate', 'artClass',
       'artCategory', 'artSize', 'artNotes', 'numberPage', 'pdfPage',
       'editionNumber', 'highlightType', 'highlightPriority', 'highlight',
       'highlightimage', 'highlightimagename', 'idMateria', 'Identifica',
       'Data', 'Ementa', 'Titulo', 'SubTitulo', 'TextoHTML', 'file_name',
       'zip_name', 'Ano', 'Mes', 'Texto', 'Total_palavras', 'TextoTratado',
       'TextoTratado_TotalPalavras', 'Texto_sem_nomes', 'Nomes',
       'TextoTratadoSemNomes', 'TextoTratadoSemNomes_TotalPalavras',
       'TextoTratadoSemNomesLemmatized',
       'TextoTratadoSemNomesLemmatized_TotalPalavras', 'USE_Embeddings1',
       'USE_Embeddings4', 'SBERT_Embeddings1', 'SBERT_Embeddings4',
       'USE_Embeddings1_PCA', 'USE_Embeddings4_PCA', 'SBERT_Embeddings1_PCA',
       'SBERT_Embeddings4_PCA', 'USE_Embeddings1_TSNE', 'USE_Embeddings4_TSNE',
       'SBERT_Embeddings1_TSNE', 'SBERT_Embeddings4_TSNE',
       'USE_Embeddi

## Funções

In [ ]:
def clusterizar_e_plotar_dbscan(df, coluna, eps, min_samples, sufixo_coluna="dbscan", sufixo_grafico=""):
    """
    Aplica o DBSCAN em dados reduzidos (ex.: t-SNE), calcula métricas de avaliação e gera o gráfico dos clusters.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os dados.
        coluna (str): Nome da coluna que contém os embeddings reduzidos (ex.: t-SNE).
        eps (float): Valor do parâmetro eps para o DBSCAN.
        min_samples (int): Valor do parâmetro min_samples para o DBSCAN.

    Retorno:
        pd.DataFrame: DataFrame atualizado com os clusters atribuídos pelo DBSCAN.
    """
    # Extração dos embeddings
    embeddings_2d = np.vstack(df[coluna])

    # Aplicação do DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
    clusters = dbscan.fit_predict(embeddings_2d)

    # Adicionar clusters ao DataFrame
    cluster_column_name = f"{coluna}_{sufixo_coluna}"
    df[cluster_column_name] = clusters

    # Filtrar clusters válidos (excluindo ruído)
    mask_valid_clusters = clusters != -1  # Ignorar rótulos de ruído (-1)
    valid_embeddings = embeddings_2d[mask_valid_clusters]
    valid_clusters = clusters[mask_valid_clusters]

    # Verificar se há clusters válidos suficientes
    if len(set(valid_clusters)) > 1:  # Garantir que há mais de um cluster válido
        silhouette_avg = silhouette_score(valid_embeddings, valid_clusters)
        davies_bouldin = davies_bouldin_score(valid_embeddings, valid_clusters)
        calinski_harabasz = calinski_harabasz_score(valid_embeddings, valid_clusters)

        print("\n--- Métricas de Avaliação ---")
        print(f"Índice de Silhueta (clusters válidos): {silhouette_avg:.6f}")
        print(f"Índice de Davies-Bouldin (clusters válidos): {davies_bouldin:.6f}")
        print(f"Índice de Calinski-Harabasz (clusters válidos): {calinski_harabasz:.2f}")
    else:
        print("Não há clusters suficientes para calcular as métricas.")

    # Proporção de ruído
    num_points = len(clusters)
    num_noise = np.sum(clusters == -1)
    noise_ratio = num_noise / num_points
    print(f"Proporção de ruído: {noise_ratio:.2%}")

    # Total de clusters válidos (excluindo ruído)
    total_clusters_excl_noise = len(set(clusters) - {-1})  # Remove o cluster de ruído (-1)
    print(f"Total de clusters válidos (excluindo ruído): {total_clusters_excl_noise}")

    # Visualização dos clusters
    print("Gerando gráfico dos clusters...")
    plt.figure(figsize=(10, 6))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=clusters, cmap='viridis', s=5)

    # Adicionar sufixo ao título do gráfico, se fornecido
    grafico_titulo = f"Clusterização com DBSCAN (eps={eps}, min_samples={min_samples})"
    if sufixo_grafico:
        grafico_titulo = f"Clusterização com DBSCAN (eps={eps}, min_samples={min_samples}) - {sufixo_grafico}"
    plt.title(grafico_titulo)
    
    plt.xlabel("Dimensão 1")
    plt.ylabel("Dimensão 2")
    plt.colorbar(label="Cluster")
    plt.grid(True)
    plt.show()

In [ ]:
def processar_dbscan_e_clusterizar(df, coluna_embeddings, eps_values, min_samples_values):
    """
    Processa o DBSCAN para várias combinações de parâmetros, identifica a melhor configuração,
    realiza a clusterização e gera a visualização.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os dados.
        coluna_embeddings (str): Nome da coluna do DataFrame com os embeddings a serem clusterizados.
        eps_values (list): Lista de valores de eps a serem testados.
        min_samples_values (list): Lista de valores de min_samples a serem testados.

    Retorno:
        pd.DataFrame: DataFrame atualizado com a clusterização da melhor configuração.
    """
    # Gerar combinações de DBSCAN e coletar métricas
    print("Iniciando o processamento para várias combinações de eps e min_samples...")
    df_resultados = []
    
    embeddings = np.vstack(df[coluna_embeddings])
    for eps in eps_values:
        for min_samples in min_samples_values:
            print(f"Processando DBSCAN para eps={eps}, min_samples={min_samples}...")
            dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
            clusters = dbscan.fit_predict(embeddings)

            # Filtrar clusters válidos
            mask_valid_clusters = clusters != -1
            valid_embeddings = embeddings[mask_valid_clusters]
            valid_clusters = clusters[mask_valid_clusters]

            # Calcular métricas se houver mais de um cluster válido
            if len(set(valid_clusters)) > 1:
                silhouette = silhouette_score(valid_embeddings, valid_clusters)
                davies_bouldin = davies_bouldin_score(valid_embeddings, valid_clusters)
                calinski_harabasz = calinski_harabasz_score(valid_embeddings, valid_clusters)
            else:
                silhouette = davies_bouldin = calinski_harabasz = np.nan

            # Calcular proporção de ruído
            num_points = len(clusters)
            num_noise = np.sum(clusters == -1)
            noise_ratio = (num_noise / num_points) * 100  # Em percentual

            # Armazenar resultados
            df_resultados.append({
                'eps': eps,
                'min_samples': min_samples,
                'Silhouette Score': silhouette,
                'Davies-Bouldin Score': davies_bouldin,
                'Calinski-Harabasz Score': calinski_harabasz,
                'ruido_percent': round(noise_ratio, 2),
                'clusters_validos': len(set(valid_clusters)) - (1 if -1 in clusters else 0)
            })

    # Converter resultados para DataFrame
    df_resultados = pd.DataFrame(df_resultados)
   
    return df_resultados


## 1) Embedding Universal Sentence Encoder (USE)

###  1.1) Sem tratamento do texto

#### 1.1.1) Sem redução de dimensionalidade

In [ ]:
vColuna = 'USE_Embeddings1'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.7, 5, sufixo_coluna="dbscan")

#### 1.1.2) Redução com PCA

In [ ]:
vColuna = 'USE_Embeddings1_PCA'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
# clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.7, 3, sufixo_coluna="dbscan", sufixo_grafico="PCA")

#### 1.1.3) Redução com t-SNE

In [ ]:
vColuna = 'USE_Embeddings1_TSNE'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.7, 5, sufixo_coluna="dbscan", sufixo_grafico="t-SNE")

#### 1.1.4) Redução com PCA + t-SNE

In [ ]:
vColuna = 'USE_Embeddings1_PCA_TSNE'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.7, 5, sufixo_coluna="dbscan", sufixo_grafico="PCA + t-SNE")

#### 1.1.5) Redução com UMAP

In [ ]:
vColuna = 'USE_Embeddings1_UMAP'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.4, 0.5], min_samples_values = [50, 60, 70, 80])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.4, 70, sufixo_coluna="dbscan", sufixo_grafico="UMAP")

###  1.2) Com tratamento do texto

#### 1.2.1) Sem redução de dimensionalidade

In [ ]:
vColuna = 'USE_Embeddings4'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.3, 15, sufixo_coluna="dbscan")

#### 1.2.2) Redução com PCA

In [ ]:
vColuna = 'USE_Embeddings4_PCA'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
# clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.3, 15, sufixo_coluna="dbscan", sufixo_grafico="PCA")

#### 1.2.3) Redução com t-SNE

In [ ]:
vColuna = 'USE_Embeddings4_TSNE'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.7, 5, sufixo_coluna="dbscan", sufixo_grafico="t-SNE")

#### 1.2.4) Redução com PCA + t-SNE

In [ ]:
vColuna = 'USE_Embeddings4_PCA_TSNE'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.7, 5, sufixo_coluna="dbscan", sufixo_grafico="PCA + t-SNE")

#### 1.2.5) Redução com UMAP

In [ ]:
vColuna = 'USE_Embeddings4_UMAP'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.2, 0.3, 0.4, 0.5], min_samples_values = [10, 15, 20, 30])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.4, 30, sufixo_coluna="dbscan", sufixo_grafico="UMAP")

## 2) Embedding Sentence-BERT (SBERT)

###  2.1) Sem tratamento do texto

#### 2.1.1) Sem redução de dimensionalidade

In [ ]:
vColuna = 'SBERT_Embeddings1'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.5, 5, sufixo_coluna="dbscan")

#### 2.1.2) Redução com PCA

In [ ]:
vColuna = 'SBERT_Embeddings1_PCA'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
# clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.3, 15, sufixo_coluna="dbscan", sufixo_grafico="UMAP")

#### 2.1.3) Redução com t-SNE

In [ ]:
vColuna = 'SBERT_Embeddings1_TSNE'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.7, 5, sufixo_coluna="dbscan", sufixo_grafico="t-SNE")

#### 2.1.4) Redução com PCA + t-SNE

In [ ]:
vColuna = 'SBERT_Embeddings1_PCA_TSNE'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.7, 5, sufixo_coluna="dbscan", sufixo_grafico="PCA + t-SNE")

#### 2.1.5) Redução com UMAP

In [ ]:
vColuna = 'SBERT_Embeddings1_UMAP'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.2, 0.3, 0.4, 0.5], min_samples_values = [30, 40, 50, 60])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.3, 40, sufixo_coluna="dbscan", sufixo_grafico="UMAP")

###  2.2) Com tratamento do texto

#### 2.2.1) Sem redução de dimensionalidade

In [ ]:
vColuna = 'SBERT_Embeddings4'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna,  eps_values = [0.2, 0.3], min_samples_values = [20, 30, 40])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.3, 30, sufixo_coluna="dbscan")

#### 2.2.2) Redução com PCA

In [ ]:
vColuna = 'SBERT_Embeddings4_PCA'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna,  eps_values = [0.3, 0.5, 0.7], min_samples_values = [5, 10, 15])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
# clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.3, 40, sufixo_coluna="dbscan", sufixo_grafico="PCA")

#### 2.2.3) Redução com t-SNE

In [ ]:
vColuna = 'SBERT_Embeddings4_TSNE'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna,  eps_values = [0.7, 0.8, 0.9], min_samples_values = [3, 5, 8])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.8, 3, sufixo_coluna="dbscan", sufixo_grafico="t-SNE")

#### 2.2.4) Redução com PCA + t-SNE

In [ ]:
vColuna = 'SBERT_Embeddings4_PCA_TSNE'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna,  eps_values = [0.4, 0.5, 0.7], min_samples_values = [2, 3, 5, 8])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.5, 2, sufixo_coluna="dbscan", sufixo_grafico="PCA + t-SNE")

#### 2.2.5) Redução com UMAP

In [ ]:
vColuna = 'SBERT_Embeddings4_UMAP'
resultado_dbscan = processar_dbscan_e_clusterizar(df_portarias_mgi, vColuna, eps_values = [0.2, 0.3, 0.4, 0.5], min_samples_values = [30, 40, 50, 60])

# Exibir os resultados
resultado_dbscan.head(100)

In [ ]:
clusterizar_e_plotar_dbscan(df_portarias_mgi, vColuna, 0.2, 30, sufixo_coluna="dbscan", sufixo_grafico="UMAP")

In [ ]:
# Salva saída parcial o arquivo completo com novas colunas
df_portarias_mgi.to_parquet(f'./saida/06_2_DOU{secao}_portarias_mgi_tratado_NER_lematizado_embeddingsUSE_SBERT_Reducao_Kmeans_DBSCAN.parquet', engine='pyarrow', index=False)